# Virtual Cell Model - MD - Simulations

In [ ]:
###Working directory###

try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd -q $workdir

In [ ]:
### Import packages ###
import mdtraj as md
import numpy as np
import matplotlib as mpl, matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 100%; }
    div#maintoolbar-container { width: 100%; }
</style>
"""))
plt.rcParams.update({'font.size':14,'legend.frameon':True,'figure.figsize':[12,8],'xtick.major.size':7,'ytick.major.size':7,'legend.labelspacing':1})

# First, fix the structure with pdbfixer! - Only the first time!

# Create the box with GROMACS - Only the first time!

In [ ]:
# We used GROMACS v. GROMACS/4.5.5 but any other version will be equivalent

!editconf -f 1ZFU.pdb  -bt cubic -box 8 8 8 -angles 90 90 90 -c -d 1 -o 1ZFU_box.pdb #&> /dev/null

# Solution conditions and parameters

In [ ]:
ns = 250 # nano seconds
dt = 2e-6 # integration step ns
###############################

Ions = {'Cl':{ 'pH': [2], 'r':[2.4, 2.6, 2.8, 3.0], 'FF':'amber99sb_Cl.xml', 'eq_steps':10000, 'steps':round(ns/dt), 'c_ions':7},
        'F': { 'pH': [2], 'r':[2.4, 2.6, 2.8, 3.0], 'FF':'amber99sb_F.xml', 'eq_steps':10000, 'steps':round(ns/dt), 'c_ions':7}}

# Energy minimization - GPUs

In [ ]:
# Generate input for energy minimizations

for name, prop in Ions.items():
    %mkdir $name
    %cd -q $name
    for pH in prop ['pH']:
        %mkdir $pH
        %cd -q $pH
        for r in prop ['r']:
            %mkdir $r
            %cd -q $r
            for g in range(1):
                %mkdir $g
                %cd -q $g
                openmm_script="""from __future__ import print_function
from simtk.openmm.app import *
from simtk.openmm import * 
import simtk.openmm as mm
from simtk.unit import *
from simtk import unit
from sys import stdout
import mdtraj as md
import numpy as np


input_pdb = '/home/marpoli/snic2020-6-15/Marco/1_Virtual_Cell_Model/Plectasin/MD/1ZFU_box.pdb'
workdir =  '/home/marpoli/snic2020-6-15/Marco/1_Virtual_Cell_Model/Plectasine/MD/'                
                
### Input ###
pdb = PDBFile(input_pdb) # For simulation
file = md.load(input_pdb)# To select atoms

### Force Field and Water Model ###
forcefield = ForceField('"""+workdir+'/'+str(prop['FF'])+"""', 'spce.xml')

###Modeller object###  replace "pdb" with "modeller" in system, simulation and simulation.context
modeller = Modeller(pdb.topology, pdb.positions)

### Protein protonation state according to the selected pH ###
modeller.addHydrogens(forcefield, pH="""+str(pH)+""") 

### Fill the box with water molecules, and neutralize it ### 
modeller.addSolvent(forcefield,  neutralize=True, negativeIon='"""+str(name)+'-'"""')

### Create system
system = forcefield.createSystem(modeller.topology, 
                                 nonbondedMethod=PME, 
                                 nonbondedCutoff=1.0*unit.nanometers, 
                                 constraints=HBonds, 
                                 ewaldErrorTolerance=0.0005, 
                                 removeCMMotion=True)
                                 
    
### Fixing the backbone positions ###
backbone = file.topology.select('backbone')
backbone_list = []
for i in backbone:
    backbone_list.append(i)
for a in backbone_list:
    system.setParticleMass(int(a), 1e10)

### Integrator ###
integrator = mm.LangevinIntegrator(298.15*kelvin, 1.0/unit.picoseconds, 4.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

### Platform ###
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0,1'}

### Start Simulation ###
simulation = Simulation(modeller.topology, system, integrator, platform, properties)
simulation.context.setPositions(modeller.positions)

### Minimization energy ###
print('Minimizing...')
simulation.minimizeEnergy()
print('Done!')

### Save file ###
positions = simulation.context.getState(getPositions=True, enforcePeriodicBox=False).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('min.pdb', 'w'), keepIds=True)


                """
                file_handle = open('min.py','w')
                file_handle.write(openmm_script)
                file_handle.close()

                %cd -q ../
            %cd -q ../
        %cd -q ../
    %cd -q ../

# Submit minimizations to cluster

In [ ]:
for name, prop in Ions.items():
    %cd -q $name
    for pH in prop ['pH']:
        %cd -q $pH
        for r in prop ['r']:
            %cd -q $r
            for g in range(1):
                %cd -q $g
                aurora_script="""#!/bin/bash
#SBATCH -p gpu
#SBATCH --gres=gpu:2
#SBATCH --mem-per-cpu=500
#SBATCH -N 1
#SBATCH -A ...
#SBACTH --tasks-per-node=10
#SBACTH -n 10

#
# job time, change for what your job requires
#SBATCH -t 01:00:00
#
# job name
#SBATCH -J """+str(name)+"""_m
#
# filenames stdout and stderr - customise, include %j
#SBATCH -e min_err
#SBATCH -o min_out

cd $SLURM_SUBMIT_DIR

module purge
module load CUDA
python min.py"""

                file_handle = open('min.sh','w')
                file_handle.write(aurora_script)
                file_handle.close()
                run = 'min.sh'
                
                #!sbatch $run
                %cd -q ../
            %cd -q ../
        %cd -q ../
    %cd -q ../

# Equilibration and production runs - GPUs

In [ ]:
for name, prop in Ions.items():
    #%mkdir $name
    %cd -q $name
    for pH in prop ['pH']:
        #%mkdir $pH
        %cd -q $pH
        for r in prop ['r']:
            #%mkdir $r
            %cd -q $r
            for g in range(1):
                #%mkdir $g
                %cd -q $g
                steps = prop['steps']
                #print(steps)
                eq_steps = prop['eq_steps']
                openmm_script="""from __future__ import print_function
from simtk.openmm.app import *
from simtk.openmm import * 
import simtk.openmm as mm
from simtk.unit import *
from simtk import unit
from sys import stdout
import mdtraj as md
import numpy as np

workdir =  '/home/marpoli/snic2020-6-15/Marco/1_Virtual_Cell_Model/Plectasine/MD/' 
                
### Input ###
pdb = PDBFile('min.pdb') # For simulation
file = md.load('min.pdb') # To select atoms

### Force Field and Water Model ###
forcefield = ForceField('"""+workdir+'/'+str(prop['FF'])+"""', 'spce.xml')

###Modeller object###  replace "pdb" with "modeller" in system, simulation and simulation.context
modeller = Modeller(pdb.topology, pdb.positions)

### Protein protonation state according to the selected pH ###
modeller.addHydrogens(forcefield, pH="""+str(pH)+""") 

### Create system. Constraining the hbonds allowed for a larger integration time step, about 4/3 fs for Langevin dynamics but it crashed. So keep 2 fs
system = forcefield.createSystem(modeller.topology, 
                                 nonbondedMethod=PME, 
                                 nonbondedCutoff=1.0*unit.nanometers, 
                                 constraints=HBonds, 
                                 ewaldErrorTolerance=0.0005, 
                                 removeCMMotion=True)


protein_CI = file.topology.select('protein or name """+str(name)+"""') # Select protein and c.ions for reporters

### Add force to counterions ###
Cions = file.topology.select('name """+str(name)+"""')
force = CustomExternalForce('100*max(0, r-"""+str(r)+""")^2; r=sqrt((x-4.0)^2+(y-4.0)^2+(z-4.0)^2)')
system.addForce(force)
for i in Cions[:"""+str(prop['c_ions'])+"""]:
    force.addParticle(int(i), [])

### Fixing the backbone positions ###
backbone = file.topology.select('backbone')
backbone_list = []
for i in backbone:
    backbone_list.append(i)
for a in backbone_list:
    system.setParticleMass(int(a), 1e10)

# Barostat
system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 298.15*unit.kelvin, 25))    

### Integrator ###
integrator = mm.LangevinIntegrator(298.15*kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

### Platform ###
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0,1'}

### Start Simulation ###
simulation = Simulation(modeller.topology, system, integrator, platform, properties)
simulation.context.setPositions(modeller.positions)


### Load checkpoint ###

# Load the checkpoint
#with open('checkpoint_1.chk', 'rb') as f:
#    simulation.context.loadCheckpoint(f.read())

### Temperature equilibration ###
simulation.context.setVelocitiesToTemperature(298.15*kelvin)
print('Equilibrating...')
simulation.step("""+str(int(eq_steps))+""")

### Reporters ####
dcd = md.reporters.DCDReporter('run.dcd', 1000, atomSubset=protein_CI)
simulation.reporters.append(dcd)

simulation.reporters.append(PDBReporter('run.pdb', """+str(int(steps))+"""))


simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True, 
    time=True, potentialEnergy=False, kineticEnergy=False, totalEnergy=True, 
    temperature=True, progress=True, 
    remainingTime=True, speed=True, totalSteps="""+str(int(steps))+""", separator='\t'))

print('Running Production...')
simulation.step("""+str(int(steps))+""")

### Save checkpoint ###
with open('checkpoint_1.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())

print('Done!')"""
                file_handle = open('run.py','w')
                file_handle.write(openmm_script)
                file_handle.close()

                %cd -q ../
            %cd -q ../
        %cd -q ../
    %cd -q ../

# Aurora GPU

In [ ]:
for name, prop in Ions.items():
    %cd -q $name
    for pH in prop ['pH']:
        %cd -q $pH
        for r in prop ['r']:
            %cd -q $r
            for g in range(1):
                %cd -q $g
                aurora_script="""#!/bin/bash
#SBATCH -p gpu
#SBATCH --gres=gpu:2
#SBATCH --mem-per-cpu=500
#SBATCH -N 1
#SBATCH -A lu2021-2-39
#SBACTH --tasks-per-node=10
#SBACTH -n 10

#
# job time, change for what your job requires
#SBATCH -t 168:00:00
#
# job name
#SBATCH -J """+str(name)+"""_r
#
# filenames stdout and stderr - customise, include %j
#SBATCH -e run_err
#SBATCH -o run_out

cd $SLURM_SUBMIT_DIR

module purge
module load CUDA
python run.py"""

                file_handle = open('run.sh','w')
                file_handle.write(aurora_script)
                file_handle.close()
                run = 'run.sh'
                
                #!sbatch $run
                %cd -q ../
            %cd -q ../
        %cd -q ../
    %cd -q ../
